# How to edit morphologies

`Jaxley` provides functionality to edit morphologies. For example, it can delete parts of a morphology.

## Deleting parts of a morphology

```python
from jaxley.io.graph import swc_to_graph, from_graph

fname = "data/morph.swc"
graph = swc_to_graph(fname)

# manipulate the graph
ids = nx.get_node_attributes(graph, "id")
ids = {k: v for k, v in ids.items() if v != 4}  # Apical dendrite has `id=4`.
graph = nx.subgraph(graph, ids).copy()

cell = from_graph(graph, ncomp=2)
```

## More explanation

All of the above is achieved by using `Jaxley`'s "graph-backend", which is based on `networkX`. As such, you can modify `Jaxley` morphologies with any feature of the [networkX](https://networkx.org) toolkit.

For more details on `Jaxley`'s graph-backend, see [this tutorial](https://jaxley.readthedocs.io/en/latest/tutorials/08_importing_morphologies.html).